# Grid navigation for different input representations

In [1]:
colab = False

if colab:
    from google.colab import drive
    drive.mount('/content/gdrive')
    !git clone https://github.com/deepmind/pycolab.git
    !git clone https://github.com/nicoladainese96/RelationalDeepRL.git
    !pip install pycolab

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
if colab: 
    import sys
    sys.path.insert(0, 'RelationalDeepRL')

from RelationalModule import ActorCritic, ControlActorCritic, CoordActorCritic, OheActorCritic
from Utils import train_agent_sandbox as train
from Utils import test_env
from Utils import utils
from importlib import reload

In [3]:
reload(test_env)

<module 'Utils.test_env' from '/home/nicola/Nicola_unipd/MasterThesis/RelationalDeepRL/Utils/test_env.py'>

In [4]:
def plot_loss(losses, x='', y='', t='', xlog=False, ylog=False):
    """Generic plotting function"""
    episodes = np.arange(len(losses)) + 1
    plt.figure(figsize=(8,6))
    plt.plot(episodes, losses)
    plt.xlabel(x, fontsize=16)
    plt.ylabel(y, fontsize=16)
    plt.title(t, fontsize=16)
    if xlog:
        plt.xscale('log')
    if ylog:
        plt.yscale('log')

# Coordinate State Representation

Simplest representation to learn, already providing high-level features.

state = (x_agent, y_agent, x_goal, y_goal, near_borders)

In [ ]:
# Variable parameters
X = 10
Y = 10
initial = [0,0]
goal = [6,7]

# All game parameters
game_params = dict(x=X, y=Y, initial=initial, goal=goal, max_steps=100, 
                   greyscale_state=False, return_coord=True, R0=-0.01)

In [ ]:
HPs = dict(observation_space = 5, lr=0.0005, gamma=0.9, TD=True, 
                 twin=True, tau=0.1, n_steps=40, H=1e-1, hiddens=[64,32,16])

if colab:
    HPs['device'] = 'cuda'
else:
    HPs['device'] = 'cpu'
    
# Relational Agent
agent = CoordActorCritic.CoordA2C(action_space = 4, **HPs)

In [ ]:
# see if everything works before starting the real training cycle
env = test_env.Sandbox(**game_params)
rewards, log_probs, distributions, states, done, bootstrap = train.play_episode(agent, env, max_steps=50)

In [ ]:
print("rewards: ", rewards.shape)
print("log_probs: ", len(log_probs))
print("distributions: ", len(distributions))
print("states: ", states.shape)
print("done: ", done.shape)
print("bootstrap: ", bootstrap.shape)

In [ ]:
print("rewards: \n", rewards)
print("log_probs: \n", log_probs)
print("distributions: \n", distributions)
print("states: \n", states)
print("done: \n", done)
print("bootstrap: \n", bootstrap)

In [ ]:
agent.update(rewards, log_probs, distributions, states, done, bootstrap)

# Training cycle

Reward: $R_0$ for each step except the one that ends the episode going to the goal. No penalty agains walls.

Now one should use a random agent and an optimal one to see what is the window of rewards better than random and less or equal to the best.

In [ ]:
reload(train)

In [ ]:
MAX_STEPS = 100
game_params = dict(x=X, y=Y, initial=[0,0], goal=[9,9], max_steps=MAX_STEPS, 
                   greyscale_state=False, return_coord=True, R0=0)

In [ ]:
%%time
results = train.train_sandbox(agent, game_params, n_episodes = 5000,
                              max_steps=MAX_STEPS, return_agent=True, random_init=True)

In [ ]:
score, asymptotic_score, asymptotic_std, trained_agent, time_profile, losses, steps_to_solve = results

In [ ]:
%matplotlib inline

In [ ]:
plt.figure(figsize=(8,6))
n_epochs = np.arange(100, len(score))
average_score = np.array([np.mean(score[i:i+100]) for i in range(len(score)-100)])
plt.plot(n_epochs, average_score, alpha=0.9)
plt.title("Performance", fontsize=16)
plt.xlabel("Number of epochs", fontsize=16)
plt.ylabel("Total reward", fontsize=16)
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
n_epochs = np.arange(100, len(score))
average_steps = np.array([np.mean(steps_to_solve[i:i+100]) for i in range(len(score)-100)])
plt.plot(n_epochs, average_steps, alpha=0.9)
plt.title("Performance", fontsize=16)
plt.xlabel("Number of epochs", fontsize=16)
plt.ylabel("Steps to solve", fontsize=16)
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
n_epochs = np.arange(len(score))
plt.plot(n_epochs, losses['critic_losses'], alpha=0.9, label='critic loss')
plt.title("Performance", fontsize=16)
plt.xlabel("Number of epochs", fontsize=16)
plt.ylabel("Loss", fontsize=16)
plt.legend(fontsize=13)
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
n_epochs = np.arange(len(score))
plt.plot(n_epochs, losses['actor_losses'], alpha=0.9, label='actor loss')
plt.plot(n_epochs, np.array(losses['entropies']), alpha=0.9, label='entropy term')
plt.title("Performance", fontsize=16)
plt.xlabel("Number of epochs", fontsize=16)
plt.ylabel("Total reward", fontsize=16)
plt.legend(fontsize=13)
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
n_epochs = np.arange(len(score))
#plt.plot(n_epochs, losses['actor_losses'], alpha=0.9, label='actor loss')
plt.plot(n_epochs, -np.array(losses['entropies']), alpha=0.9, label='entropy term')
plt.title("Entropy", fontsize=16)
plt.xlabel("Number of epochs", fontsize=16)
plt.ylabel("Entropy loss", fontsize=16)
plt.legend(fontsize=13)
plt.show()

**How to read and interpret:** Entropy is higher for policies that are highly stochastic and zero for those that are deterministic. The spikes toward smaller entropies are probably due to strong feedbacks (either positive or negative ones). If we have |A| possible actions, the maximum value of the entropy is:
$$S = \sum_{a=1}^4 - \frac{1}{|A|}log\left( \frac{1}{|A|} \right) = - log\left( \frac{1}{|A|} \right) = +log(|A|) \approx 1.4 $$
Given that the actor updates have order of the hundred, we should use a coupling constant much higher that the one currently used $(10^{-2})$. The other problem is that scaling the reward would leave the entropy unchanged, thus if the actor loss changes of many orders of magnitude during training, we must be aware that entropy regularization might become a real problem.

## Test

In [ ]:
env = test_env.Sandbox(**game_params)
R = 0
test_episodes = 100
max_steps = 30
tot_steps = 0

for i in range(test_episodes):
    rewards, log_probs, distributions, states, done, bootstrap = train.play_episode(trained_agent, env, max_steps=max_steps)
    #print("Solved in %d steps"%len(rewards))
    R += rewards.sum()
    tot_steps += len(rewards)
print("Average reward: ", R/test_episodes)
print("Steps to solve: ", tot_steps/test_episodes)

In [ ]:
reload(utils)

In [ ]:
import matplotlib.pyplot as plt
import time
%matplotlib notebook

# Variable parameters
X = 10
Y = 10
initial = [0,9]
goal = [5,4]

# All game parameters
#game_params = dict(x=X, y=Y, initial=initial, goal=goal, max_steps=100, greyscale_state=True)
game_params = dict(x=X, y=Y, initial=initial, goal=goal, max_steps=100, 
                   greyscale_state=False, return_coord=True)

env = test_env.Sandbox(**game_params)
#utils.render(trained_agent, env, save=False)
render(trained_agent, env, save=False)

# OneHotEncoded State Representation

In [ ]:
# Variable parameters
X = 10
Y = 10
initial = [0,0]
goal = [6,7]
MAX_STEPS = 100

game_params = dict(x=X, y=Y, initial=initial, goal=goal, max_steps=MAX_STEPS, 
                   greyscale_state=True, return_ohe=True)

In [ ]:
HPs = dict(lr=0.0005, gamma=0.9, TD=True, twin=True, tau=0.1, n_steps=40, H=1e-1)

if colab:
    HPs['device'] = 'cuda'
else:
    HPs['device'] = 'cpu'
    
# Relational Agent
agent = OheActorCritic.OheA2C(action_space = 4, map_size=X, **HPs)

In [ ]:
# see if everything works before starting the real training cycle
env = test_env.Sandbox(**game_params)
rewards, log_probs, distributions, states, done, bootstrap = train.play_episode(agent, env, max_steps=MAX_STEPS)

In [ ]:
print("rewards: ", rewards.shape)
print("log_probs: ", len(log_probs))
print("distributions: ", len(distributions))
print("states: ", states.shape)
print("done: ", done.shape)
print("bootstrap: ", bootstrap.shape)

## Training cycle

In [ ]:
%%time
results = train.train_sandbox(agent, game_params, n_episodes = 5000,
                              max_steps=MAX_STEPS, return_agent=True, random_init=True)

In [ ]:
score, asymptotic_score, asymptotic_std, trained_agent, time_profile, losses, steps_to_solve = results

## Reloading partially trained agent

In [ ]:
load_dir = "Results/Sandbox/"
queries = ['ohe', 'sandbox']
train_dict = utils.load_session(load_dir, queries)

game_params = train_dict['game_params']
HPs = train_dict['HPs']
score = train_dict['score']
keywords = train_dict['keywords']
ID = keywords[-1]

# Load agent 
if colab:
    trained_agent = torch.load(load_dir+"agent_"+ID)
else:
    trained_agent = torch.load(load_dir+"agent_"+ID, map_location=torch.device('cpu'))
    trained_agent.device = 'cpu'

In [ ]:
%%time
results = train.train_sandbox(agent, game_params, n_episodes = 5000,
                              max_steps=MAX_STEPS, return_agent=True, random_init=True)

In [ ]:
score, asymptotic_score, asymptotic_std, trained_agent, time_profile, losses, steps_to_solve = results

In [ ]:
import matplotlib.pyplot as plt
import time
%matplotlib notebook

# Variable parameters

X = 10
Y = 10
initial = [0,0]
goal = [9,9]
MAX_STEPS = 100

game_params = dict(x=X, y=Y, initial=initial, goal=goal, max_steps=MAX_STEPS, 
                   greyscale_state=True, return_ohe=True)

env = test_env.Sandbox(**game_params)
utils.render(trained_agent, env, save=False)

In [ ]:
save = True
keywords = ['control', '10by10',str(len(score))+"-episodes","200-steps","NQBK-pt2"] # example

if colab and save:
    %cd ~
    parent_dir = "/content/gdrive/My Drive/Colab Notebooks/"
    save_dir  = "RelationalTrained/"
    %cd "{parent_dir}"
    !mkdir "{save_dir}"
    ID = utils.save_session(save_dir, keywords, game_params, HPs, score)
    torch.save(trained_agent, save_dir+"agent_"+ID)
elif save:
    save_dir = 'Results/Sandbox/'
    ID = utils.save_session(save_dir, keywords, game_params, HPs, score)
    torch.save(trained_agent, save_dir+"agent_"+ID)
else:
    print("Nothing saved")
    pass

## Relational agent with OHE state representation

In [5]:
# Variable parameters
X = 10
Y = 10
initial = [0,0]
goal = [6,7]
MAX_STEPS = 200

game_params = dict(x=X, y=Y, initial=initial, goal=goal, max_steps=MAX_STEPS, 
                   greyscale_state=True, return_ohe=True)

In [6]:
HPs = dict(lr=0.0005, gamma=0.9, TD=True, twin=True, tau=0.1, n_steps=40, H=1e-1)

if colab:
    HPs['device'] = 'cuda'
else:
    HPs['device'] = 'cpu'
    
# Relational Agent
agent = ActorCritic.BoxWorldA2C(action_space = 4, **HPs)

In [7]:
# see if everything works before starting the real training cycle
env = test_env.Sandbox(**game_params)
rewards, log_probs, distributions, states, done, bootstrap = train.play_episode(agent, env, max_steps=MAX_STEPS)

In [8]:
print("rewards: ", rewards.shape)
print("log_probs: ", len(log_probs))
print("distributions: ", len(distributions))
print("states: ", states.shape)
print("done: ", done.shape)
print("bootstrap: ", bootstrap.shape)

rewards:  (200,)
log_probs:  200
distributions:  200
states:  (201, 3, 12, 12)
done:  (200,)
bootstrap:  (200,)


In [9]:
agent.update(rewards, log_probs, distributions, states, done, bootstrap)

(0.12117810547351837, -0.11549694091081619, -0.026653388515114784)

In [ ]:
%%time
results = train.train_sandbox(agent, game_params, n_episodes = 15000,
                              max_steps=MAX_STEPS, return_agent=True, random_init=True)

In [ ]:
score, asymptotic_score, asymptotic_std, trained_agent, time_profile, losses, steps_to_solve = results

In [ ]:
plt.figure(figsize=(8,6))
n_epochs = np.arange(100, len(score))
average_steps = np.array([np.mean(steps_to_solve[i:i+100]) for i in range(len(score)-100)])
plt.plot(n_epochs, average_steps, alpha=0.9)
plt.title("Performance", fontsize=16)
plt.xlabel("Number of epochs", fontsize=16)
plt.ylabel("Steps to solve", fontsize=16)
plt.show()

In [ ]:
save = True
keywords = ['relational','ohe', '10by10',str(len(score))+"-episodes","200-steps"] # example

if colab and save:
    %cd ~
    parent_dir = "/content/gdrive/My Drive/Colab Notebooks/"
    save_dir  = "RelationalTrained/"
    %cd "{parent_dir}"
    !mkdir "{save_dir}"
    ID = utils.save_session(save_dir, keywords, game_params, HPs, score, steps_to_solve)
    torch.save(trained_agent, save_dir+"agent_"+ID)
elif save:
    save_dir = 'Results/Sandbox/'
    ID = utils.save_session(save_dir, keywords, game_params, HPs, score, steps_to_solve)
    torch.save(trained_agent, save_dir+"agent_"+ID)
else:
    print("Nothing saved")
    pass

# Embeddied State Representation

In [ ]:
# Variable parameters
X = 10
Y = 10
initial = [0,0]
goal = [6,7]

game_params = dict(x=X, y=Y, initial=initial, goal=goal, max_steps=100, greyscale_state=True)

## Scripted critic

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F 
from torch.distributions import Categorical

from RelationalModule.AC_networks import ControlCritic #custom module

debug = False 

class CriticAgentTD():
    """
    Advantage Actor-Critic RL agent for BoxWorld environment described in the paper
    Relational Deep Reinforcement Learning.
    
    Notes
    -----
    * Always uses 2 separate networks for the critic,one that learns from new experience 
      (student/critic) and the other one (critic_target/teacher)that is more conservative 
      and whose weights are updated through an exponential moving average of the weights 
      of the critic, i.e.
          target.params = (1-tau)*target.params + tau* critic.params
    * In the case of Monte Carlo estimation the critic_target is never used
    * Possible to use twin networks for the critic and the critic target for improved 
      stability. Critic target is used for updates of both the actor and the critic and
      its output is the minimum between the predictions of its two internal networks.
      
    """ 
    
    def __init__(self, lr, gamma, TD=True, twin=False, tau = 1., 
                 H=1e-2, n_steps = 1, device='cpu', **box_net_args):
        
        self.gamma = gamma
        self.lr = lr
        
        self.TD = TD
        self.twin = twin 
        self.tau = tau
        self.H = H
        self.n_steps = n_steps
        
        self.critic = ControlCritic(twin, **box_net_args)
        
        if self.TD:
            self.critic_trg = ControlCritic(twin, target=True, **box_net_args)

            # Init critic target identical to critic
            for trg_params, params in zip(self.critic_trg.parameters(), self.critic.parameters()):
                trg_params.data.copy_(params.data)
            
        self.critic_optim = torch.optim.Adam(self.critic.parameters(), lr=lr)
        
        self.device = device 
        self.critic.to(self.device)
        if self.TD:
            self.critic_trg.to(self.device)
        
        if debug:
            print("="*10 +" A2C HyperParameters "+"="*10)
            print("Discount factor: ", self.gamma)
            print("Learning rate: ", self.lr)
            print("Temporal Difference learning: ", self.TD)
            print("Twin networks: ", self.twin)
            print("Update critic target factor: ", self.tau)
            if self.TD:
                print("n_steps for TD: ", self.n_steps)
            print("Device used: ", self.device)
            print("\n\n"+"="*10 +" A2C Architecture "+"="*10)
            print("Critic architecture: \n",self.critic)
            print("Critic target architecture: ")
            if self.TD:
                print(self.critic_trg)
            else:
                print("Not used")
    
    def update_TD(self, rewards, states, done, bootstrap=None):   
        
        ### Compute n-steps rewards, states, discount factors and done mask ###
        
        n_step_rewards = self.compute_n_step_rewards(rewards)
        if debug:
            print("n_step_rewards.shape: ", n_step_rewards.shape)
            print("rewards.shape: ", rewards.shape)
            print("n_step_rewards: ", n_step_rewards)
            print("rewards: ", rewards)
            print("bootstrap: ", bootstrap)
                
        if bootstrap is not None:
            done[bootstrap] = False 
        if debug:
            print("done.shape: (before n_steps)", done.shape)
            print("done: (before n_steps)", done)
        
        old_states = torch.tensor(states[:-1].astype(int)).to(self.device)

        new_states, Gamma_V, done = self.compute_n_step_states(states, done)
        new_states = torch.tensor(new_states.astype(int)).to(self.device)

        if debug:
            print("done.shape: (after n_steps)", done.shape)
            print("Gamma_V.shape: ", Gamma_V.shape)
            print("done: (after n_steps)", done)
            print("Gamma_V: ", Gamma_V)
            print("old_states.shape: ", old_states.shape)
            print("new_states.shape: ", new_states.shape)
            
        ### Wrap variables into tensors ###
        
        done = torch.LongTensor(done.astype(int)).to(self.device)
    
        n_step_rewards = torch.tensor(n_step_rewards).float().to(self.device)
        Gamma_V = torch.tensor(Gamma_V).float().to(self.device)
        
        ### Update critic and then actor ###
        critic_loss = self.update_critic_TD(n_step_rewards, new_states, old_states, done, Gamma_V)
        #actor_loss, entropy = self.update_actor_TD(n_step_rewards, log_probs, distributions, new_states, old_states, done, Gamma_V)
        
        return critic_loss#, actor_loss, entropy
    
    def update_critic_TD(self, n_step_rewards, new_states, old_states, done, Gamma_V):
        
        # Compute loss 
        if debug: print("Updating critic...")
        with torch.no_grad():
            V_trg = self.critic_trg(new_states).squeeze()
            if debug:
                print("V_trg.shape (after critic): ", V_trg.shape)
            V_trg = (1-done)*Gamma_V*V_trg + n_step_rewards
            if debug:
                print("V_trg.shape (after sum): ", V_trg.shape)
            V_trg = V_trg.squeeze()
            if debug:
                print("V_trg.shape (after squeeze): ", V_trg.shape)
                print("V_trg.shape (after squeeze): ", V_trg)
            
        if self.twin:
            V1, V2 = self.critic(old_states)
            if debug:
                print("V1.shape: ", V1.squeeze().shape)
                print("V1: ", V1)
            loss1 = 0.5*F.mse_loss(V1.squeeze(), V_trg)
            loss2 = 0.5*F.mse_loss(V2.squeeze(), V_trg)
            loss = loss1 + loss2
        else:
            V = self.critic(old_states).squeeze()
            if debug: 
                print("V.shape: ",  V.shape)
                print("V: ",  V)
            loss = F.mse_loss(V, V_trg)
        
        # Backpropagate and update
        
        self.critic_optim.zero_grad()
        loss.backward()
        self.critic_optim.step()
        
        # Update critic_target: (1-tau)*old + tau*new
        
        for trg_params, params in zip(self.critic_trg.parameters(), self.critic.parameters()):
                trg_params.data.copy_((1.-self.tau)*trg_params.data + self.tau*params.data)
        
        return loss.item()
    
    def compute_n_step_rewards(self, rewards):
        """
        Computes n-steps discounted reward padding with zeros the last elements of the trajectory.
        This means that the rewards considered are AT MOST n, but can be less for the last n-1 elements.
        """
        T = len(rewards)
        
        # concatenate n_steps zeros to the rewards -> they do not change the cumsum
        r = np.concatenate((rewards,[0 for _ in range(self.n_steps)])) 
        
        Gamma = np.array([self.gamma**i for i in range(r.shape[0])])
        
        # reverse everything to use cumsum in right order, then reverse again
        Gt = np.cumsum(r[::-1]*Gamma[::-1])[::-1]
        
        G_nstep = Gt[:T] - Gt[self.n_steps:] # compute n-steps discounted return
        
        Gamma = Gamma[:T]
        
        assert len(G_nstep) == T, "Something went wrong computing n-steps reward"
        
        n_steps_r = G_nstep / Gamma
        
        return n_steps_r
    
    def compute_n_step_states(self, states, done):
        """
        Computes n-steps target states (to be used by the critic as target values together with the
        n-steps discounted reward). For last n-1 elements the target state is the last one available.
        Adjusts also the `done` mask used for disabling the bootstrapping in the case of terminal states
        and returns Gamma_V, that are the discount factors for the target state-values, since they are 
        n-steps away (except for the last n-1 states, whose discount is adjusted accordingly).
        
        Return
        ------
        new_states, Gamma_V, done: arrays with first dimension = len(states)-1
        """
        
        # Compute indexes for (at most) n-step away states 
        
        n_step_idx = np.arange(len(states)-1) + self.n_steps
        diff = n_step_idx - len(states) + 1
        mask = (diff > 0)
        n_step_idx[mask] = len(states) - 1
        
        # Compute new states
        
        new_states = states[n_step_idx]
        
        # Compute discount factors
        
        pw = np.array([self.n_steps for _ in range(len(new_states))])
        pw[mask] = self.n_steps - diff[mask]
        Gamma_V = self.gamma**pw
        
        # Adjust done mask
        
        mask = (diff >= 0)
        done[mask] = done[-1]
        
        return new_states, Gamma_V, done


In [ ]:
def play_optimal(env):
    state = env.reset(random_init = True)
    
    rewards = []
    done = []
    states = [state]
    bootstrap = []
    
    while True:
        action = env.get_optimal_action()
        
        new_state, reward, terminal, info = env.step(action) 
        states.append(new_state)
        rewards.append(reward)
        done.append(terminal)
        
        if terminal is True and 'TimeLimit.truncated' in info:
            bootstrap.append(True)
        else:
            bootstrap.append(False) 
            
        if terminal:
            break
            
        state = new_state
    
    return np.array(rewards), np.array(states), np.array(done), np.array(bootstrap)

In [ ]:
reload(test_env)

In [ ]:
def train_critic_TD(HPs, n_episodes = 100):
    
    # Create environment
    env = test_env.Sandbox(10, 10, [0,0], [9,9], max_steps=50)
    
    critic = CriticAgentTD(**HPs)
    
    losses = []
    for e in range(n_episodes):
        rewards, states, done, bootstrap = play_optimal(env)
        loss = critic.update_TD(rewards, states, done, bootstrap)
        losses.append(loss)
        
        if (e+1)%10 == 0:
            print("Episode %d - loss: %.4f"%(e+1, np.mean(losses[-10:])))

    return critic, losses

In [ ]:
HPs = dict(lr=0.0005, gamma=0.99, TD=True, twin=True, tau=0.01, n_steps=1,
               H=1., vocab_size = 4, n_dim=3, n_features=16, n_heads=4, linear_size=12)

In [ ]:
critic, losses = train_critic_TD(HPs, 5000)

In [ ]:
x = "Number of episodes"
y = "Squared error of the prediction [log]"
t = "Critic trained through optimal policy and TD"
plot_loss(losses, x, y, t, ylog=True)

In [ ]:
import copy
MAP_SIZE = 10 + 2 # because of the border
goal = (9+1,9+1) # because of the border
state = np.full([MAP_SIZE,MAP_SIZE], 0)
values = np.full([MAP_SIZE,MAP_SIZE], 0).astype(float)
state[goal] = 2

for s in range(MAP_SIZE**2):
    x, y = s//MAP_SIZE, s%MAP_SIZE
    new_state = copy.deepcopy(state)
    new_state[x,y] = 1
    new_state = torch.LongTensor(new_state).unsqueeze(0)
    with torch.no_grad():
        V1, V2 = critic.critic(new_state)
        V = torch.min(V1,V2)
    values[x,y] = V.squeeze().numpy()

In [ ]:
def get_real_value_map(env, gamma):
    distances = np.zeros((env.boundary[0], env.boundary[1] ))
    for x in range(env.boundary[0]):
        for y in range(env.boundary[1]):
            d = env.dist_to_goal([x,y])
            distances[x,y] = d
    distances = distances.flatten() -1
    real_values = gamma**distances
    real_values[-1] = 1
    return real_values.reshape(env.boundary[0], env.boundary[1])

In [ ]:
def value_map(critic, env):
    MAP_SIZE = env.boundary[0] 
    goal = (env.goal[0]+1, env.goal[1]+1)
    gamma = critic.gamma
    
    state = np.full([MAP_SIZE+2,MAP_SIZE+2], 0)
    values = np.full([MAP_SIZE,MAP_SIZE], 0).astype(float)
    state[goal] = 2
    # adding borders
    state[0,:] = 3
    state[-1,:] = 3
    state[:,0] = 3
    state[:,-1] = 3
    
    for s in range(MAP_SIZE**2):
        x, y = s//MAP_SIZE, s%MAP_SIZE
        new_state = copy.deepcopy(state)
        new_state[x+1,y+1] = 1
        new_state = torch.LongTensor(new_state).unsqueeze(0)
        with torch.no_grad():
            V1, V2 = critic.critic(new_state)
            V = torch.min(V1,V2)
        values[x,y] = V.squeeze().numpy()
    
    plt.figure(figsize=(15,6))

    plt.subplot(121)
    values[-1,-1] = 1
    plt.imshow(values)
    plt.xticks([])
    plt.yticks([])
    plt.title("Value map", fontsize=16)
    plt.colorbar()
    
    real_values = get_real_value_map(env, gamma)
    plt.subplot(122)
    plt.imshow(real_values)
    plt.xticks([])
    plt.yticks([])
    plt.title("Real values", fontsize=16)
    plt.colorbar()
    
    plt.figure(figsize=(7,6))
    diff = values-real_values
    diff[-1,-1] = 0
    plt.imshow(diff)
    #plt.imshow(real_values)
    plt.xticks([])
    plt.yticks([])
    plt.title("Difference from real value", fontsize=16)
    plt.colorbar()
    

In [ ]:
env = test_env.Sandbox(10, 10, [0,0], [9,9], max_steps=50)
value_map(critic, env)

## Scripted actor

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F 
from torch.distributions import Categorical

from RelationalModule.AC_networks import ControlActor

debug = False

class ActorAgentTD():
    """
    Advantage Actor-Critic RL agent for BoxWorld environment described in the paper
    Relational Deep Reinforcement Learning.
    
    Notes
    -----
    * Always uses 2 separate networks for the critic,one that learns from new experience 
      (student/critic) and the other one (critic_target/teacher)that is more conservative 
      and whose weights are updated through an exponential moving average of the weights 
      of the critic, i.e.
          target.params = (1-tau)*target.params + tau* critic.params
    * In the case of Monte Carlo estimation the critic_target is never used
    * Possible to use twin networks for the critic and the critic target for improved 
      stability. Critic target is used for updates of both the actor and the critic and
      its output is the minimum between the predictions of its two internal networks.
      
    """ 
    
    def __init__(self, action_space, lr, gamma, TD=True, twin=False, tau = 1., 
                 H=1e-2, n_steps = 1, device='cpu', **box_net_args):
        """
        Parameters
        ----------
        action_space: int
            Number of (discrete) possible actions to take
        lr: float in [0,1]
            Learning rate
        gamma: float in [0,1]
            Discount factor
        TD: bool (default=True)
            If True, uses Temporal Difference for the critic's estimates
            Otherwise uses Monte Carlo estimation
        twin: bool (default=False)
            Enables twin networks both for critic and critic_target
        tau: float in [0,1] (default = 1.)
            Regulates how fast the critic_target gets updates, i.e. what percentage of the weights
            inherits from the critic. If tau=1., critic and critic_target are identical 
            at every step, if tau=0. critic_target is unchangable. 
            As a default this feature is disabled setting tau = 1, but if one wants to use it a good
            empirical value is 0.005.
        H: float (default 1e-2)
            Entropy multiplicative factor in actor's loss
        n_steps: int (default=1)
            Number of steps considered in TD update
        device: str in {'cpu','cuda'}
            Select if training agent with cpu or gpu. 
            FIXME: At the moment is gpu is present, it MUST use the gpu.
        **box_net_args: dict (optional)
            Dictionary of {'key':value} pairs valid for BoxWorldNet.
            Valid keys:
                in_channels: int (default 3)
                    Number of channels of the input image (e.g. 3 for RGB)
                n_kernels: int (default 24)
                    Number of features extracted for each pixel
                vocab_size: int (default 116)
                    Range of integer values of the raw pixels
                n_dim: int (default 3)
                    Embedding dimension for each pixel channel (1 channel for greyscale, 
                    3 for RGB)
                n_features: int (default 256)
                    Number of linearly projected features after positional encoding.
                    This is the number of features used during the Multi-Headed Attention
                    (MHA) blocks
                n_heads: int (default 4)
                    Number of heades in each MHA block
                n_attn_modules: int (default 2)
                    Number of MHA blocks
                n_linears: int (default 4)
                    Number of fully-connected layers after the FeaturewiseMaxPool layer
        """
        
        self.gamma = gamma
        self.lr = lr
        
        self.n_actions = action_space
        self.TD = TD
        self.twin = twin 
        self.tau = tau
        self.H = H
        self.n_steps = n_steps
        
        self.actor = ControlActor(action_space, **box_net_args)
       

        self.actor_optim = torch.optim.Adam(self.actor.parameters(), lr=lr)
            
     
        self.device = device 
        self.actor.to(self.device) 
        
    def get_action(self, state, return_log=False):
        log_probs = self.forward(state)
        dist = torch.exp(log_probs)
        probs = Categorical(dist)
        action =  probs.sample().item()
        if return_log:
            return action, log_probs.view(-1)[action], dist
        else:
            return action
    
    def forward(self, state):
        """
        Makes a tensor out of a numpy array state and then forward
        it with the actor network.
        
        Parameters
        ----------
        state: array of int
            Shape (episode_len, in_channels, lin_size, lin_size)
            Or    (in_channels, lin_size, lin_size)
        """
        state = torch.from_numpy(state.astype(int)).to(self.device)
        log_probs = self.actor(state)
        return log_probs
    
    def critic(self, state, env):
        """Implements optimal critic given the sandbox environment."""
        AGENT_COLOR = 1
        distances = []
        for s in state:
            #print("s: ", s)
            shape = s.shape
            #print("shape: ", shape)
            s = s.numpy().reshape(shape[1], shape[2])
            x, y = np.where(s==AGENT_COLOR)
            #print("x and y: ", x, y)
            s_decoded = [x, y]
            #print("s_decoded: ", s_decoded)
            #print("\n\tState %d decoded in (%d, %d)"%(s, s_decoded[0], s_decoded[1]))
            d = env.dist_to_goal(s_decoded)
            #print("\tGoal in (%d,%d)"%(env.goal[0], env.goal[1]))
            #print("\tDistance from goal: ", d)
            distances.append(d)
        distances = np.array(distances) -1
        #print("distances.shape: ", distances.shape)
        #print("Distances: ", distances)
        #print("gamma*dist: ", (self.gamma**distances).shape)
        values = torch.tensor(self.gamma**distances).float()
        
        #print("Values: ", values,'\n')
        return values
    
    def update_TD(self, env, rewards, log_probs, distributions, states, done, bootstrap=None):   
        
        ### Compute n-steps rewards, states, discount factors and done mask ###
        
        n_step_rewards = self.compute_n_step_rewards(rewards)
        if debug:
            print("n_step_rewards.shape: ", n_step_rewards.shape)
            print("rewards.shape: ", rewards.shape)
            print("n_step_rewards: ", n_step_rewards)
            print("rewards: ", rewards)
            #print("bootstrap: ", bootstrap)
                
        if bootstrap is not None:
            done[bootstrap] = False 
        #if debug:
            #print("done.shape: (before n_steps)", done.shape)
            #print("done: (before n_steps)", done)
        
        old_states = torch.tensor(states[:-1].astype(int)).to(self.device)

        new_states, Gamma_V, done = self.compute_n_step_states(states, done)
        new_states = torch.tensor(new_states.astype(int)).to(self.device)

        if debug:
            print("done.shape: (after n_steps)", done.shape)
            print("Gamma_V.shape: ", Gamma_V.shape)
            print("done: (after n_steps)", done)
            print("Gamma_V: ", Gamma_V)
            print("old_states.shape: ", old_states.shape)
            print("new_states.shape: ", new_states.shape)
            
        ### Wrap variables into tensors ###
        
        done = torch.LongTensor(done.astype(int)).to(self.device)
        #if debug: print("log_probs: ", log_probs)
        log_probs = torch.stack(log_probs).to(self.device)
        #if debug: print("log_probs: ", log_probs)
        distributions = torch.stack(distributions, axis=1).to(self.device)
        # Regularize terms with 0 probability (if that happens)
        mask = (distributions == 0).nonzero()
        distributions[:,mask] = 1e-5
        
        n_step_rewards = torch.tensor(n_step_rewards).float().to(self.device)
        Gamma_V = torch.tensor(Gamma_V).float().to(self.device)
        
        ### Update critic and then actor ###
        #critic_loss = self.update_critic_TD(n_step_rewards, new_states, old_states, done, Gamma_V)
        actor_loss = self.update_actor_TD(env, n_step_rewards, log_probs, distributions, new_states, old_states, done, Gamma_V)
        
        return actor_loss
    

    def update_actor_TD(self, env, n_step_rewards, log_probs, distributions, new_states, old_states, done, Gamma_V):
        
        # Compute gradient 
        if debug: print("Updating actor...")
        with torch.no_grad():
            V_pred = self.critic(old_states, env).squeeze()
            V_trg = self.critic(new_states, env).squeeze()
            V_trg = (1-done)*Gamma_V*V_trg + n_step_rewards
            
        A = V_trg - V_pred
        policy_gradient = - log_probs*A
        if debug:
            print("V_trg.shape: ",V_trg.shape)
            print("V_trg: ", V_trg)
            print("V_pred.shape: ",V_pred.shape)
            print("V_pred: ", V_pred)
            print("A.shape: ", A.shape)
            print("A: ", A)
            print("policy_gradient.shape: ", policy_gradient.shape)
            print("policy_gradient: ", policy_gradient)
        policy_grad = torch.mean(policy_gradient)
        if debug: print("policy_grad: ", policy_grad)
        entropy = self.H*torch.mean(distributions*torch.log(distributions))
        loss = policy_grad + entropy
        if debug: print("Actor loss: ", loss)
        
        # Backpropagate and update
    
        self.actor_optim.zero_grad()
        loss.backward()
        self.actor_optim.step()
        
        return policy_grad.item()
    
    def compute_n_step_rewards(self, rewards):
        """
        Computes n-steps discounted reward padding with zeros the last elements of the trajectory.
        This means that the rewards considered are AT MOST n, but can be less for the last n-1 elements.
        """
        T = len(rewards)
        
        # concatenate n_steps zeros to the rewards -> they do not change the cumsum
        r = np.concatenate((rewards,[0 for _ in range(self.n_steps)])) 
        
        Gamma = np.array([self.gamma**i for i in range(r.shape[0])])
        
        # reverse everything to use cumsum in right order, then reverse again
        Gt = np.cumsum(r[::-1]*Gamma[::-1])[::-1]
        
        G_nstep = Gt[:T] - Gt[self.n_steps:] # compute n-steps discounted return
        
        Gamma = Gamma[:T]
        
        assert len(G_nstep) == T, "Something went wrong computing n-steps reward"
        
        n_steps_r = G_nstep / Gamma
        
        return n_steps_r
    
    def compute_n_step_states(self, states, done):
        """
        Computes n-steps target states (to be used by the critic as target values together with the
        n-steps discounted reward). For last n-1 elements the target state is the last one available.
        Adjusts also the `done` mask used for disabling the bootstrapping in the case of terminal states
        and returns Gamma_V, that are the discount factors for the target state-values, since they are 
        n-steps away (except for the last n-1 states, whose discount is adjusted accordingly).
        
        Return
        ------
        new_states, Gamma_V, done: arrays with first dimension = len(states)-1
        """
        
        # Compute indexes for (at most) n-step away states 
        
        n_step_idx = np.arange(len(states)-1) + self.n_steps
        diff = n_step_idx - len(states) + 1
        mask = (diff > 0)
        n_step_idx[mask] = len(states) - 1
        
        # Compute new states
        
        new_states = states[n_step_idx]
        
        # Compute discount factors
        
        pw = np.array([self.n_steps for _ in range(len(new_states))])
        pw[mask] = self.n_steps - diff[mask]
        Gamma_V = self.gamma**pw
        
        # Adjust done mask
        
        mask = (diff >= 0)
        done[mask] = done[-1]
        
        return new_states, Gamma_V, done
    

In [ ]:
def play_episode(agent, env):
    # Reset environment (start of an episode)
    state = env.reset(random_init=True)
    
    states = [state]
    rewards = []
    log_probs = []
    distributions = []
    done = []
    bootstrap = []
    
        
        
    while True:
        action, log_prob, distr = agent.get_action(state, return_log=True)
        new_state, reward, terminal, info = env.step(action) # gym standard step's output
        

        states.append(new_state)
        rewards.append(reward)
        log_probs.append(log_prob)
        distributions.append(distr)
        done.append(terminal)
        
        if terminal is True and 'TimeLimit.truncated' in info:
            bootstrap.append(True)
        else:
            bootstrap.append(False) 
            
        if terminal:
            break
            
        state = new_state
       
    return  np.array(rewards), log_probs, distributions, np.array(states), np.array(done), np.array(bootstrap)

In [ ]:
def train_actor_TD(actor, env, n_episodes = 100):
    losses = []
    performances = []
    steps_to_solve = []
    for e in range(n_episodes):
        rewards, log_probs, distributions, states, done, bootstrap = play_episode(actor, env)
        loss = actor.update_TD(env, rewards, log_probs, distributions, states, done, bootstrap)
        losses.append(loss)
        performances.append(rewards.sum())
        steps_to_solve.append(len(rewards))
        if (e+1)%10 == 0:
            print("Episode %d - reward: %.2f -  steps to solve: %.2f"%(e+1, np.mean(performances[-10:]), np.mean(steps_to_solve[-10:])))

    return actor, performances, losses, steps_to_solve

In [ ]:
HPs = dict(lr=0.0003, gamma=0.99, TD=True, twin=True, tau=1., n_steps=1,
               H=1., vocab_size = 4, n_dim=3, n_features=64, n_heads=4, linear_size=7)

In [ ]:
env = test_env.Sandbox(5, 5, [0,0], [4,4], max_steps=50)
action_space = 4
actor = ActorAgentTD(action_space, **HPs)

In [ ]:
%%time
actor_TD, score, actor_loss_TD, steps_to_solve = train_actor_TD(actor, env, 1000)

In [ ]:
x = "Number of episodes"
y = "Actor loss"
t = "Actor trained through optimal critic"
plot_loss(actor_loss_TD, x, y, t, ylog=False)

In [ ]:
x = "Number of episodes"
y = "Total reward"
t = "Actor trained through optimal critic"
window = 500
moving_score =  np.array([np.mean(score[i:i+window]) for i in range(len(score)-window)])
plot_loss(moving_score, x, y, t, ylog=False)

In [ ]:
from Utils import utils

In [ ]:
%matplotlib notebook
utils.render(actor_TD, x=5, y=5, goal=[4,4])

In [ ]:
%matplotlib inline

In [ ]:
save = False
keywords = ['relational', 'sandbox',str(len(score))+"-episodes","100-steps"] # example

if colab and save:
    %cd ~
    parent_dir = "/content/gdrive/My Drive/Colab Notebooks/"
    save_dir  = "RelationalTrained/"
    %cd "{parent_dir}"
    !mkdir "{save_dir}"
    ID = utils.save_session(save_dir, keywords, game_params, HPs, score)
    torch.save(trained_agent, save_dir+"agent_"+ID)
elif save:
    save_dir = 'Results/Sandbox/'
    ID = utils.save_session(save_dir, keywords, game_params, HPs, score)
    torch.save(trained_agent, save_dir+"agent_"+ID)
else:
    print("Nothing saved")
    pass